# Notebook - Metrics for Evaluating an Identity Verification Solution 

This notebook provides a walkthrough of the steps mentioned in the blog "Metrics for Evaluating an Identity Verification Solution"

## Step 1 - Data Prep 

1) Upload a dataset of images to an S3 bucket in a single folder. This should include all the images to be tested (both Genuine and Imposter)

2) In the same bucket, upload an image pair mapping file in CSV format. Ensure that the column headers are exactly the same as is described in Section 1 of the blog.

The resulting folder structure of your S3 bucket should look like the following:
```
S3 bucket
    │ 
    │ -image_pair_mapping_file.csv
    │
    └── dataset/
       │    -file011.jpg
       │    -file012.jpg
       │      .
       │      .

```

### Install python dependencies if necessary

In [ ]:
!pip install boto3 pandas seaborn matplotlib sklearn 

### Import libraries

In [ ]:
import boto3
import io
import pandas as pd
from io import BytesIO
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
# Initialize clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# Enter the bucket name where you uploaded the dataset in Step 1
bucket_name = "ENTER_BUCKET_NAME" # example: "my_bucket_name"
dataset_folder = "ENTER_FOLDER_NAME" # example: "dataset/"

In [ ]:
# Enter the image pair mapping (CSV) file name that you uploaded in Step 1
csv_file = "ENTER_FILE_NAME" # example: "dataset_v1.csv"

### Load the image pair mapping file - Genuine examples

In [ ]:
obj = s3.get_object(Bucket= bucket_name , Key = csv_file)
df = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8')

In [ ]:
df.loc[df['TEST'] == "Genuine"].head()

### Load the image pair mapping file - Imposter examples

In [ ]:
df.loc[df['TEST'] == "Imposter"].head()

## Step 2 - Probing the "genuine" and "imposter image pair sets

### Loop through all image pairs in dataset and get similarity score

In [ ]:
# Apply the Rekognition CompareFaces API
def compare_faces(source_file, target_file, threshold = 0):
    response=rekognition.compare_faces(SimilarityThreshold=threshold,
                                  SourceImage={'S3Object': {
                                      'Bucket': bucket_name,
                                      'Name':source_file}},
                                  TargetImage={'S3Object': {
                                      'Bucket': bucket_name,
                                      'Name':target_file}})
    
    # This notebook assumes 1 face in an image
    if len(response['FaceMatches']) == 1:
        similarity = float(response['FaceMatches'][0]['Similarity'])
        return similarity
    elif len(response['FaceMatches']) == 0:
        return float(0.0)       
    else:
        return "Images should only contain 1 face for this notebook"

In [ ]:
df_similarity = df.copy()
df_similarity["SIMILARITY"] = None
    
for index, row in df.iterrows():
    source_file = dataset_folder + row["SOURCE"]
    target_file = dataset_folder + row["TARGET"]
    response_score = compare_faces(source_file, target_file)
    df_similarity._set_value(index,"SIMILARITY", response_score)

df_similarity.head()

### BoxPlot of distribution of similarity score by test set

In [ ]:
sns.boxplot(data=df_similarity,
            x=df_similarity["SIMILARITY"],
            y=df_similarity["TEST"]).set(xlabel='Similarity Score', 
                                         ylabel=None, 
                                         title = "Similarity Score Distribution")
plt.show()

### Descriptive Statistics

In [ ]:
df_descriptive_stats = pd.DataFrame(columns=['test','count', 'min' , 'max', 'mean', 'median', 'std'])
tests = ["Genuine", "Imposter"]

for test in tests:
    count = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].count()
    mean = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].mean()
    max_ = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].max()
    min_ = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].min()
    median = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].median()
    std = df_similarity['SIMILARITY'].loc[df_similarity['TEST'] == test].std()

    new_row = {'test': test,
                'count': count,
                'min': min_,
                'max': max_,
                'mean': mean,
                'median':median,
                'std': std}
    df_descriptive_stats = df_descriptive_stats.append(new_row,ignore_index=True)

df_descriptive_stats

## Step 4 - Calculate False Match and Non-Match Rates at different similarity threshold levels

In [ ]:
similarity_thresholds = [80,85,90,95,96,97,98,99]

# create output df
df_cols = ['Similarity Threshold', 'TN' , 'FN', 'TP', 'FP', 'FNMR (%)', 'FMR (%)']
comparison_df = pd.DataFrame(columns=df_cols)

# create columns for y_actual and y_pred
df_analysis = df_similarity.copy()
df_analysis["y_actual"] = None  
df_analysis["y_pred"] = None
    
    
for threshold in similarity_thresholds:
    # Create y_pred and y_actual columns, 1 == match, 0 == no match
    for index, row in df_similarity.iterrows():
        # set y_pred
        if row["SIMILARITY"] >= threshold:
            df_analysis._set_value(index,"y_pred", 1) 
        else:
            df_analysis._set_value(index,"y_pred", 0) 

        # set y_actual
        if row["TEST"] == "Genuine":
            df_analysis._set_value(index,"y_actual", 1)
        else:
            df_analysis._set_value(index,"y_actual", 0)
    
    tn, fp, fn, tp = confusion_matrix(df_analysis['y_actual'].tolist(), df_analysis['y_pred'].tolist()).ravel()
    FNMR = fn / (tp + fn)
    FMR = fp / (tn+fp+fn+tp)
    
    new_row = {'Similarity Threshold': threshold,
                                        'TN': tn,
                                        'FN': fn,
                                        'TP': tp,
                                        'FP': fp,
                                        'FNMR (%)':FNMR,
                                        'FMR (%)': FMR}
    comparison_df = comparison_df.append(new_row,ignore_index=True)
comparison_df